In [1]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql.window import Window
import time
import shutil
from pyspark.sql.functions import current_timestamp, col, unix_timestamp

from datetime import datetime, timedelta


import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


In [3]:
# df = spark.read.format("csv") \
#     .option("header", "true") \
#     .load("gs://indicator-crypto/sma_results/BTC/*")


In [4]:
# schema = StructType([
#     StructField("BASE", StringType(), True),  
#     StructField("DATE", StringType(), True),
#     StructField("OPEN", DoubleType(), True),
#     StructField("HIGH", DoubleType(), True),
#     StructField("LOW", DoubleType(), True),
#     StructField("CLOSE", DoubleType(), True),
#     StructField("VOLUME", DoubleType(), True),
#     StructField("YEAR", IntegerType(), True),
#     StructField("MONTH", IntegerType(), True),
#     StructField("__index_level_0__", LongType(), True)
# ])
# df = spark.read.schema(schema).parquet("gs://crypto-historical-data-2/ver2/ADA/2021/*")

In [5]:
# df.show(truncate=False)

In [16]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


column_names = ["BTC", "ETH", "USDT", "USDC", "XRP", "ADA", "DOGE", "MATIC", "SOL"]

schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])

import datetime

def read_historical_data(coin):
    current_year = datetime.datetime.now().year  
    years = [str(year) for year in range(2021, current_year + 1)]  

    paths = [f"gs://crypto-historical-data-2/ver2/{coin}/{year}/*" for year in years]
    
    return (
        spark.read.schema(schema).parquet(*paths)  
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )


# Function to calculate RSI-14
def calculate_rsi(df, window_size=14):
    df = df.withColumn("price_change", F.col("CLOSE") - F.lag("CLOSE").over(Window.orderBy("DATE")))
    df = df.withColumn("gain", F.when(F.col("price_change") > 0, F.col("price_change")).otherwise(0))
    df = df.withColumn("loss", F.when(F.col("price_change") < 0, -F.col("price_change")).otherwise(0))

    window_spec = Window.orderBy("DATE").rowsBetween(-window_size + 1, 0)
    
    df = df.withColumn("avg_gain", F.avg("gain").over(window_spec))
    df = df.withColumn("avg_loss", F.avg("loss").over(window_spec))

    df = df.withColumn("rs", F.col("avg_gain") / F.col("avg_loss"))
    df = df.withColumn("RSI_14", 100 - (100 / (1 + F.col("rs"))))
    
    return df

# Function to process each coin and calculate RSI
def process_coin(coin):
    historical_data_df = read_historical_data(coin)
    combined_df = calculate_rsi(historical_data_df)

    combined_df = combined_df.select("DATE", "CLOSE", "RSI_14").orderBy("DATE", ascending=False)

    tmp_dir = f"gs://indicator-crypto/rsi_results/batch/{coin}"

    combined_df.show(10, truncate=False)
    # combined_df.write \
    #     .format("csv") \
    #     .option("header", "true") \
    #     .option("path", tmp_dir) \
    #     .mode("append") \
    #     .save()

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_coin, coin) for coin in column_names]
    for future in as_completed(futures):
        print(future.result())


In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


column_names = ["BTC", "ETH", "USDT", "USDC", "XRP", "ADA", "DOGE", "MATIC", "SOL"]

schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])

import datetime

def read_historical_data(coin):
    current_year = datetime.datetime.now().year  
    years = [str(year) for year in range(2021, current_year + 1)]  

    paths = [f"gs://crypto-historical-data-2/ver2/{coin}/{year}/*" for year in years]
    
    return (
        spark.read.schema(schema).parquet(*paths)  
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )


# Function to calculate RSI-14
def calculate_rsi(df, window_size=14):
    df = df.withColumn("price_change", F.col("CLOSE") - F.lag("CLOSE").over(Window.orderBy("DATE")))
    df = df.withColumn("gain", F.when(F.col("price_change") > 0, F.col("price_change")).otherwise(0))
    df = df.withColumn("loss", F.when(F.col("price_change") < 0, -F.col("price_change")).otherwise(0))

    window_spec = Window.orderBy("DATE").rowsBetween(-window_size + 1, 0)
    
    df = df.withColumn("avg_gain", F.avg("gain").over(window_spec))
    df = df.withColumn("avg_loss", F.avg("loss").over(window_spec))

    df = df.withColumn("rs", F.col("avg_gain") / F.col("avg_loss"))
    df = df.withColumn("RSI_14", 100 - (100 / (1 + F.col("rs"))))
    
    return df

# Function to process each coin and calculate RSI
def process_coin(coin):
    historical_data_df = read_historical_data(coin)
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
    
    historical_data_df = historical_data_df.withColumn(f"SMA_5", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-4, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_10", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-9, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_20", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-19, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_50", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-49, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_100", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-99, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_200", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-199, 0)))

    historical_data_df = historical_data_df.select("DATE", "CLOSE", "SMA_5", "SMA_10", "SMA_20", "SMA_50", "SMA_100", "SMA_200").orderBy("DATE", ascending=False)

    tmp_dir = f"gs://indicator-crypto/rsi_results/batch/{coin}"

    historical_data_df.show(10, truncate=False)
    # combined_df.write \
    #     .format("csv") \
    #     .option("header", "true") \
    #     .option("path", tmp_dir) \
    #     .mode("append") \
    #     .save()

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_coin, coin) for coin in column_names]
    for future in as_completed(futures):
        print(future.result())


24/12/13 17:47:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 17:47:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 17:47:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 17:47:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 17:47:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 17:47:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 1

+-------------------+--------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|DATE               |CLOSE   |SMA_5             |SMA_10            |SMA_20            |SMA_50            |SMA_100            |SMA_200            |
+-------------------+--------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|2024-12-13 00:00:00|1.093891|1.066615          |1.1318774         |1.0954164499999999|0.74523258        |0.54955729         |0.47053314499999993|
|2024-12-12 00:00:00|1.12327 |1.0867578         |1.1417370999999998|1.0916719         |0.73034076        |0.54180338         |0.46735368999999993|
|2024-12-11 00:00:00|1.088143|1.1043524000000002|1.1496267999999998|1.0766584         |0.7151573600000001|0.53393468         |0.4640473399999999 |
|2024-12-10 00:00:00|1.022986|1.1327206         |1.1487824999999998|1.0622862499999999|0.7006085         |0.52636425  

24/12/13 17:47:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 17:47:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|DATE               |CLOSE  |SMA_5             |SMA_10            |SMA_20            |SMA_50            |SMA_100           |SMA_200           |
+-------------------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|2024-12-13 00:00:00|2.31984|2.3285379999999996|2.389802          |2.0811355         |1.2602742         |0.9092971         |0.7216790499999999|
|2024-12-12 00:00:00|2.33782|2.385396          |2.4090219999999998|2.0388735000000002|1.2243854         |0.8916717         |0.7127198499999997|
|2024-12-11 00:00:00|2.39499|2.4403259999999998|2.447825          |1.9844175         |1.188303          |0.8739705000000001|0.7037387499999999|
|2024-12-10 00:00:00|2.37267|2.4471399999999996|2.403606          |1.9199179999999998|1.1512952         |0.8554936000000001|0.6944432999